## Backtesting 

### Store Metrics/Plot Visualisations here

In [12]:
import sys
from pathlib import Path
import pandas as pd
import json
from datetime import datetime
import matplotlib.pyplot as plt

# sys.path.append('..')
from strategies.mean_reversion import MeanReversionStrat, Trade
from utilities.api_client import APIClient
from strategies.indicators import rsi, bb, atr, std_dev

# Load config
with open('../config.json', 'r') as f:
    config = json.load(f)

# Initialize backtest parameters
INITIAL_BALANCE_USDT = 10000  # Starting with 10k USDT
TRADING_FEES = 0.001  # 0.1% maker/taker fee on Binance



In [24]:
class Backtest:
    def __init__(self, strategy: MeanReversionStrat):
        self.strategy = strategy
        self.equity_curve = [strategy.env.get_equity(position=strategy.pos, pos_size=strategy.pos_size)]

    def run(self):
        #make for loop until no more data left with update period function
        while self.strategy.env.get_current_data() is not None:
            signal = self.strategy.gen_signal()

            # yea this is stupid but for readability
            if signal != "flat":
                trade = self.strategy.enter_position(signal)
                self.strategy.save_trade(trade, 'backtesting/tradetest.csv')

            self.equity_curve.append(self.strategy.env.get_equity(position=self.strategy.pos, pos_size=self.strategy.pos_size))

    def plot_equity_curve(self):
        equity_df = pd.DataFrame(self.equity_curve)
        plt.plot(equity_df['timestamp'], equity_df['equity'])
        plt.show()

### Historical Data Retrieval

First, we connect to the $\texttt{ccxt}$ API to access historical OHLCV data of our chosen pairs.

In [16]:
from keys import API_KEY, API_SECRET

client = APIClient(config, API_KEY, API_SECRET)
client.exchange.enableRateLimit = True
client.exchange.options['adjustForTimeDifference'] = True
historical_data = client.fetch_ohlcv_df('BTC/USDT')
display(historical_data)

,timestamp,open,high,low,close,volume
0,2024-08-29,59034.90,61166.99,58713.09,59359.01,27020.907430
1,2024-08-30,59359.00,59944.07,57701.10,59123.99,28519.321950
2,2024-08-31,59123.99,59462.38,58744.00,58973.99,8798.409000
3,2024-09-01,58974.00,59076.59,57201.00,57301.86,20705.157410
4,2024-09-02,57301.77,59425.69,57128.00,59132.13,22895.014610
...,...,...,...,...,...,...
95,2024-12-02,97185.17,98130.00,94395.00,95840.62,37958.669810
96,2024-12-03,95840.61,96305.52,93578.17,95849.69,35827.322830
97,2024-12-04,95849.69,99000.00,94587.83,98587.32,43850.537280
98,2024-12-05,98587.32,104088.00,90500.00,96945.63,109921.729662


### Initialising a Mean Reversion Strategy

In [25]:



strategy = MeanReversionStrat(config='config.json',
                              env_type='backtest',
                              initial_balance=INITIAL_BALANCE_USDT,
                              data=historical_data)

first_test = Backtest(strategy)
first_test.run()
first_test.plot_equity_curve()



AttributeError: 'BacktestingEnvironment' object has no attribute 'get_equity'

In [ ]:
class Backtest:
    def __init__(self, strategy, data, initial_balance):
        self.strategy = strategy
        self.data = data
        self.balance = initial_balance
        self.positions = []
        self.trades = []
        self.equity_curve = []
        
    def run(self):
        # Iterate through each day in the data
        for i in range(len(self.data)):
            current_data = self.data.iloc[:i+1]
            if len(current_data) < 20:  # Need enough data for indicators
                continue
                
            # Get signal from strategy
            signal = self.strategy.gen_signal(current_data)
            
            # Execute trades based on signal
            if signal != 0:
                # Calculate position size based on risk management
                size = self.strategy.pos_size
                
                # Include trading fees
                fee = size * current_data.iloc[-1]['close'] * TRADING_FEES
                
                # Execute trade
                self.strategy.execute_trade(signal, current_data, size)
                
                # Update balance
                self.balance -= fee
            
            # Record equity at this point
            total_equity = self.balance
            if self.strategy.pos != 0:
                # Add unrealized P&L if we have an open position
                current_price = current_data.iloc[-1]['close']
                total_equity += self.strategy.pos_size * current_price
                
            self.equity_curve.append({
                'timestamp': current_data.iloc[-1]['timestamp'],
                'equity': total_equity
            })
            
    def get_results(self):
        equity_df = pd.DataFrame(self.equity_curve)
        
        # Calculate basic metrics
        total_return = (equity_df['equity'].iloc[-1] - INITIAL_BALANCE_AUD) / INITIAL_BALANCE_AUD
        
        # Calculate drawdown
        equity_df['peak'] = equity_df['equity'].cummax()
        equity_df['drawdown'] = (equity_df['peak'] - equity_df['equity']) / equity_df['peak']
        max_drawdown = equity_df['drawdown'].max()
        
        return {
            'total_return': total_return,
            'max_drawdown': max_drawdown,
            'num_trades': len(self.strategy.trades),
            'equity_curve': equity_df
        }

# Fetch historical data
client = APIClient(config, "", "")  # Empty strings for API keys since we're just backtesting
historical_data = client.fetch_ohlcv_df('BTC/AUD')

# Initialize strategy
strategy = MeanReversionStrat(config=config, api_key="", api_secret="")

# Run backtest
backtest = Backtest(strategy, historical_data, INITIAL_BALANCE_AUD)
backtest.run()

# Get and display results
results = backtest.get_results()
print(f"Total Return: {results['total_return']*100:.2f}%")
print(f"Max Drawdown: {results['max_drawdown']*100:.2f}%")
print(f"Number of Trades: {results['num_trades']}")

# Plot equity curve
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(results['equity_curve']['timestamp'], results['equity_curve']['equity'])
plt.title('Equity Curve')
plt.xlabel('Date')
plt.ylabel('Portfolio Value (AUD)')
plt.grid(True)
plt.show()
